In [1]:
import sys
sys.path.append('..')

In [2]:
import modules.initialize as minit

minit.initialize(verbose=True)

Detected CUDA files, patching ldflags
Emitting ninja build file /home/alexey/torch_experiments/mytorchcudamodules/notebooks/../csrc/build/build.ninja...
Building extension module fused_mix_prec_layer_norm_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module fused_mix_prec_layer_norm_cuda...
ninja: no work to do.


In [3]:
from modules import MixedFusedLayerNorm
import torch

In [4]:
device = torch.device('cuda')

In [5]:
sz = [2, 4096, 4096]
layer = MixedFusedLayerNorm(sz[-1]).to(device)
input = torch.zeros(sz).to(device).unsqueeze(0)
input = input.unsqueeze(0)

out = layer(input)
print('forward', out)
print('backward', out.backward(torch.randn_like(out)))

forward tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]]], device='cuda:0',
       grad_fn=<FusedLayerNormAffineFunctionBackward>)
backward None


In [6]:
input = torch.randn(sz).to(device).unsqueeze(0)


In [7]:
%%time
for i in range(1000):
    out = layer(input)
    out.backward(torch.randn_like(out))
torch.cuda.synchronize()

CPU times: user 976 ms, sys: 817 ms, total: 1.79 s
Wall time: 1.79 s


In [8]:
%%time
with torch.cuda.amp.autocast():
    for i in range(1000):
        out = layer(input)
        out.backward(torch.randn_like(out))
torch.cuda.synchronize()

CPU times: user 1.09 s, sys: 699 ms, total: 1.79 s
Wall time: 1.79 s


### Baseline

In [9]:
import torch, torch.nn as nn
sz = [2, 4096, 4096]
device='cuda:0'
layer = nn.LayerNorm(sz[-1]).to(device)
input = torch.zeros(sz).to(device).unsqueeze(0)
input = input.unsqueeze(0)

out = layer(input)
print('forward', out)
print('backward', out.backward(torch.randn_like(out)))

forward tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]]], device='cuda:0',
       grad_fn=<NativeLayerNormBackward0>)
backward None


In [10]:
input = torch.randn(sz).to(device).unsqueeze(0)


In [11]:
%%time
for i in range(1000):
    out = layer(input)
    out.backward(torch.randn_like(out))
torch.cuda.synchronize()

CPU times: user 679 ms, sys: 398 ms, total: 1.08 s
Wall time: 1.07 s


In [12]:
%%time
with torch.cuda.amp.autocast():
    for i in range(1000):
        out = layer(input)
        out.backward(torch.randn_like(out))
torch.cuda.synchronize()

CPU times: user 680 ms, sys: 397 ms, total: 1.08 s
Wall time: 1.07 s
